### Import necessary Libraries

In [1]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

In [3]:
CLIENT_ID = 'VCSEMYEYM2QZLNBIKTS5DIH2JAHAIATDTEIQ4LBJWDDDC0JS' # your Foursquare ID
CLIENT_SECRET = 'EWZHB1MAUAMWES53FKON1FEVJESNBRYLGRN2ON1CH3WVO4EJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

Let's again assume that I am in my quatier. So let's start by converting it to its latitude and longitude coordinates.

In [4]:
address = "Liège Carré, 4000 Liège, Belgium"

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/biosys/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


50.640538 5.5619438


In [8]:
search_query = 'restaurant'
# define URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query)
# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c180b699fb6b75152201352'},
 'response': {'venues': [{'id': '575a93af498e56fd377535af',
    'name': 'CHC | Restaurant Self Service Saint-Joseph',
    'location': {'address': 'Rue de Hesbaye 75',
     'crossStreet': 'Bâtiment W - étage 0',
     'lat': 50.647431,
     'lng': 5.555387,
     'labeledLatLngs': [{'label': 'display',
       'lat': 50.647431,
       'lng': 5.555387}],
     'distance': 896,
     'postalCode': '4000',
     'cc': 'BE',
     'city': 'Liège',
     'state': 'Liège',
     'country': 'Belgique',
     'formattedAddress': ['Rue de Hesbaye 75 (Bâtiment W - étage 0)',
      '4000 Liège',
      'Belgique']},
    'categories': [{'id': '52e81612bcbc57f1066b79f4',
      'name': 'Buffet',
      'pluralName': 'Buffets',
      'shortName': 'Buffet',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1545079657',
    'hasPerk': False},
   {'id

In [11]:
type(results)

dict

### Search for a specific venue category

#### Now, let's assume that I am intrested in Vietnamese food. So, let's define a query to search for Vietnamese food that is within 500 metres from the Boulevard D'avroy. 

In [51]:
search_query = 'Chiness'
radius = 3000
print(search_query + ' .... OK!')

Chiness .... OK!


#### Define the corresponding URL

In [52]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VCSEMYEYM2QZLNBIKTS5DIH2JAHAIATDTEIQ4LBJWDDDC0JS&client_secret=EWZHB1MAUAMWES53FKON1FEVJESNBRYLGRN2ON1CH3WVO4EJ&ll=50.640538,5.5619438&v=20180604&query=Chiness&radius=3000&limit=30'

#### Send the GET Request and examine the results

In [53]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c0f8240f594df18fb558305'},
 'response': {'venues': []}}

#### Get relevant part of JSON and transform it into a pandas dataframe

In [35]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,5429294b498e82f0e3cb0b56,Rue Du Pont d'Or,BE,Liège,Belgique,579,"[Rue Du Pont d'Or, Liège, Belgique]","[{'label': 'display', 'lat': 50.64217128499785...",50.642171,5.569745,NaN,Liège,The Italian Pasta Bar,v-1544519593
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,52d4f77a11d2a314a8db0bbb,Boulevard Raymond Poincaré 7,BE,Liège,Belgique,1515,"[Boulevard Raymond Poincaré 7, 4020 Liège, Bel...","[{'label': 'display', 'lat': 50.63413816901448...",50.634138,5.580883,4020,Liège,Espressioni Italiane,v-1544519593


#### Define information of interest and filter dataframe

In [36]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
print(filtered_columns)
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

['name', 'categories', 'location.address', 'location.cc', 'location.city', 'location.country', 'location.distance', 'location.formattedAddress', 'location.labeledLatLngs', 'location.lat', 'location.lng', 'location.postalCode', 'location.state', 'id']


#### Let's visualize the Italian restaurants that are nearby

In [37]:
dataframe_filtered.name

0    The Italian Pasta Bar
1     Espressioni Italiane
Name: name, dtype: object

In [38]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Explore a Given Venue

#### Let's explore the closest Italian restaurant -- The Italian Pasta Bar

In [42]:
venue_id = '5429294b498e82f0e3cb0b56' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5429294b498e82f0e3cb0b56?client_id=VCSEMYEYM2QZLNBIKTS5DIH2JAHAIATDTEIQ4LBJWDDDC0JS&client_secret=EWZHB1MAUAMWES53FKON1FEVJESNBRYLGRN2ON1CH3WVO4EJ&v=20180604'

#### Send GET request for result

In [44]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'attributes'])


{'id': '5429294b498e82f0e3cb0b56',
 'name': 'The Italian Pasta Bar',
 'contact': {},
 'location': {'address': "Rue Du Pont d'Or",
  'lat': 50.64217128499785,
  'lng': 5.569744704206119,
  'labeledLatLngs': [{'label': 'display',
    'lat': 50.64217128499785,
    'lng': 5.569744704206119}],
  'cc': 'BE',
  'city': 'Liège',
  'state': 'Liège',
  'country': 'Belgique',
  'formattedAddress': ["Rue Du Pont d'Or", 'Liège', 'Belgique']},
 'canonicalUrl': 'https://foursquare.com/v/the-italian-pasta-bar/5429294b498e82f0e3cb0b56',
 'categories': [{'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restaurants',
   'shortName': 'Italian',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'price': {'tier': 2, 'message': 'Moderate', 'currency': '€'},
 'likes': {'count': 0, 'groups': []},
 'dislike': False,
 'ok': False,
 'allowMenuUrlEdit'

#### Get the venue's overall rating

In [45]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [46]:
venue_id = '52d4f77a11d2a314a8db0bbb' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


#### Get the venue's tips

Create URL and send GET request. Make sure to set limit to get all tips

In [55]:
## Ecco Tips
venue_id = '5429294b498e82f0e3cb0b56'
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c0f82a44434b903c4b28da9'},
 'response': {'tips': {'count': 0, 'items': []}}}